# Пример генерации текста рекурентной сетью

In [1]:
import sys
import random
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

## Загружаем исходные данные

In [2]:
with open('/Users/aleksandr/Downloads/aml/aml_17/data/anna_karenina.txt', 'rb') as src:
    binary = src.read()
    text = binary.decode('utf8').lower()
print('corpus length:', len(text))

corpus length: 1710408


## Векторизуем текстовый корпус

In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), 
             dtype=np.bool)

y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 93
nb sequences: 570123
Vectorization...


## Строим модель

In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

Build model...


## Возвращает индекс выбранного символа для вектора вероятностей

In [5]:
def sample(preds, temperature = 1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-32) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Запускаем обучение

In [6]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Train on 570123 samples
570123/570123 [==============================] - 421s 738us/sample - loss: 1.9199

----- diversity: 0.2
----- Generating with seed: "а ним, ходя под гору по лощине и на гору"
а ним, ходя под гору по лощине и на горучно подомнала ее последно подумал он, подомнала его с ней подробности в свое последно последника с которое он все это последно подумал он с ней не понять и простовать в сомнение с ней, как он не подомал и с которое он подумал он, не потом он не подумал ее с него в себя не помошли с которой он не слушал ее с которое она не по меня не последно последно последно в свое к сережа и последно подумал он.

----- diversity: 0.5
----- Generating with seed: "а ним, ходя под гору по лощине и на гору"
а ним, ходя под гору по лощине и на горучно он выражение в сомнательно подумал по в меня не преженно подружение меня только меня в сергей иванович не больно с казалась к стал в сомнательно бессердца с пр

KeyboardInterrupt: 

In [ ]:
# остановлена итерация, т.к. обучение будет долгим. Далее предсказания на предобученной модели.

## Предсказания на предобученной модели

In [7]:
# model.load_weights('./text-lstm.hd5')
model.load_weights('/Users/aleksandr/Downloads/aml/aml_17/text-lstm.hd5')

In [8]:
start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "оезд и она вышла, первое лицо, обративше"
оезд и она вышла, первое лицо, обратившего старая последнее волосами и не так не волосами старика себе себе стал васенька столковаться и не волосами волосами в петербурге столкова последнее потом она стал по волосами старая старика столкова степана аркадьически с нее потому что она подали в себе столь совсем последнее уметь в том улыбаясь в том стали стевал в столу старую стал стали себе при нее умерить на него себе сторону столкова при

----- diversity: 0.5
----- Generating with seed: "оезд и она вышла, первое лицо, обративше"
оезд и она вышла, первое лицо, обратившего все его должна молодой и странное его ровно стала удивала на ней делая неми те умет и поступить того, что то помотрела в более не столько и подари матери. степан аркадьичем обманительное отвечал степан аркадьича возвлаком развина имения особенно в ней была разговора умета от нее потому что то при взгляду те умерил воронил подите

In [ ]:
pass